In [4]:
import requests
from lxml import etree
import pandas as pd
from multiprocessing.dummy import Pool
from bs4 import BeautifulSoup
import itertools
import gc
import time

In [5]:
# 伪装头
headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
}

In [6]:
init_url = 'https://bj.lianjia.com/ershoufang/'

# 获取所有在售二手房的房屋链接

每页显示30套房子，每个城区最多显示100页 也就是3000页房子，所以要分街区获取

In [7]:
## 获取所有城区的url
html = requests.get(init_url , headers = headers).text
selector = etree.HTML(html)
districtName = selector.xpath('//*[@id="position"]/dl[2]/dd/div[1]/div/a/text()')
districtUrl = ['https://bj.lianjia.com'+i for i in selector.xpath('//*[@id="position"]/dl[2]/dd/div[1]/div/a/@href')]
## 获取地区的url
blockUrl = [] #用来存所有的URL
blockName = [] #用来村所有地区的名字
for district in districtUrl[0:-2]:# 因为北京房屋搜索的时候是包含燕郊和大厂这俩河北地区的。
    html = requests.get(district,headers = headers)
    selector = etree.HTML(html.text)
    blockName.extend(selector.xpath('//*[@id="position"]/dl[2]/dd/div[1]/div[2]/a/text()'))
    blockUrl.extend(['https://bj.lianjia.com' + i for i in selector.xpath('//*[@id="position"]/dl[2]/dd/div[1]/div[2]/a/@href')])
blockUrl = list(set(blockUrl))

In [8]:
len(blockUrl)

239

In [6]:
# 先那一个地区的url页面，获取一共有多少房子，一共有多少页
def gethouseUrl(blockUrl):
    houseUrl = []
    res = requests.get(blockUrl , headers = headers)
    soup = BeautifulSoup(res.text , 'html.parser')
    maxhouse = int(soup.select('div[class="resultDes clear"] h2 span')[0].text)
    totalPage = json.loads(soup.select('div[class="page-box house-lst-page-box"]')[0]['page-data'])['totalPage']
    print('共有%d套房子，共%d页'%(maxhouse,totalPage))
    #获取每个地区的所有页面中房子的URL
    for page in range(1,totalPage+1):
        url = blockUrl + 'ng1nb1pg' + str(page)
        res = requests.get(url, headers = headers )
        soup = BeautifulSoup(res.text,'html.parser')
        houseUrl.extend([i['href'] for i in soup.select('li[class="clear LOGCLICKDATA"] div[class="title"] a')])
        print('\r %d/%d'%(page,totalPage) , end = '')
    return houseUrl 

 
    #房屋信息（几室几厅）
    roommainInfo = selector.xpath('/html/body/div[5]/div[2]/div[3]/div[1]/div[1]/text()')
    #楼层位置（高层/共x层）
    roomsubInfo = selector.xpath('/html/body/div[5]/div[2]/div[3]/div[1]/div[2]/text()')
    #朝向
    typemainInfo = selector.xpath('/html/body/div[5]/div[2]/div[3]/div[2]/div[1]/text()')
    #户型( 平层/跃层)装修情况(精装、简装、毛坯)
    typesubInfo = selector.xpath('/html/body/div[5]/div[2]/div[3]/div[2]/div[2]/text()')
    #建筑面积
    areamainInfo = selector.xpath('/html/body/div[5]/div[2]/div[3]/div[3]/div[1]/text()')
    #年份/楼型 (板楼/塔楼)
    areasubInfo = selector.xpath('/html/body/div[5]/div[2]/div[3]/div[3]/div[2]/text()')


In [7]:
def gethouseInfo(houseUrl):
    html = requests.get(houseUrl ,headers = headers).text
    selector = etree.HTML(html)   
    
    totalPrice = selector.xpath('/html/body/div[5]/div[2]/div[2]/span[1]/text()')    #总价
    unitPrice = selector.xpath('/html/body/div[5]/div[2]/div[2]/div[1]/div[1]/span/text()')    #单价
    communityName = selector.xpath('/html/body/div[5]/div[2]/div[4]/div[1]/a[1]/text()')    #小区名称
    districtName = selector.xpath('/html/body/div[5]/div[2]/div[4]/div[2]/span[2]/a[1]/text()')    #城区
    blockName = selector.xpath('/html/body/div[5]/div[2]/div[4]/div[2]/span[2]/a[2]/text()')    #地区
    houseRecordID = selector.xpath('/html/body/div[5]/div[2]/div[4]/div[4]/span[2]/text()')    #房屋ID
    roommainInfo = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[1]/text()')#户型
    floor = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[2]/text()') #楼层
    builtArea = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[3]/text()') #建筑面积
    roomType = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[4]/text()') #房屋类型（平层、跃层）
    useArea = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[5]/text()') #使用面积
    builtType = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[6]/text()') #建筑类型（塔楼、板楼）
    orientation = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[7]/text()') #朝向
    built = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[8]/text()') #建筑结构
    decoration = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[9]/text()')# 装修情况
    liftType = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[10]/text()') #梯户比例
    heatingMode =selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[11]/text()') #供暖方式
    lift = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[12]/text()') #配备电梯
    propertyRights = selector.xpath('//*[@id="introduction"]/div/div/div[1]/div[2]/ul/li[13]/text()') #产权年限
    houseType = selector.xpath('//*[@id="introduction"]/div/div/div[2]/div[2]/ul/li[4]/span[2]/text()') #房屋用途
    
    #项目顺序
    columns = ['houseRecordID',
                'communityName',
                'blockName',
                'districtName',
                'totalPrice',
                'builtArea',
                'useArea',
                'builtType',
                'unitPrice',
                'roommainInfo',
                'floor',
                'roomType',
                'orientation',
                'built',
                'decoration',
                'liftType',
                'heatingMode',
                'lift',
                'propertyRights',
                'houseType',
                'houseUrl'] 

    houseInfo = pd.DataFrame(dict(totalPrice=totalPrice,
                                    unitPrice=unitPrice,
                                    communityName=communityName,
                                    districtName=districtName,
                                    blockName=blockName,
                                    houseRecordID=houseRecordID,
                                    roommainInfo=roommainInfo,
                                    floor=floor,
                                    builtArea=builtArea,
                                    roomType=roomType,
                                    useArea=useArea,
                                    builtType=builtType,
                                    orientation=orientation,
                                    built=built,
                                    decoration=decoration,
                                    liftType=liftType,
                                    heatingMode=heatingMode,
                                    lift=lift,
                                    propertyRights=propertyRights,
                                    houseType=houseType,      
                                    houseUrl = [houseUrl]
                                 ),columns = columns)
   # 写入csv
   # houseInfo.to_csv('houseInfo.csv' , index = False , header = False,mode="a+")
    print('#',end='')
    return houseInfo

In [3]:
def gethouseInfo_perblock(blockUrl):   
    res = requests.get(blockUrl , headers = headers)
    soup = BeautifulSoup(res.text , 'html.parser')
    maxhouse = int(soup.select('div[class="resultDes clear"] h2 span')[0].text)
    if maxhouse!= 0:
        totalPage = json.loads(soup.select('div[class="page-box house-lst-page-box"]')[0]['page-data'])['totalPage']
        print('共有%d套房子，共%d页'%(maxhouse,totalPage))
        #获取每个地区的所有页面中房子的URL
        houseUrl = []
        for page in range(1,totalPage+1):
            url = blockUrl + 'ng1nb1pg' + str(page)###不看车位不看地下室
            res = requests.get(url, headers = headers )
            soup = BeautifulSoup(res.text,'html.parser')
            houseUrl.extend([i['href'] for i in soup.select('li[class="clear LOGCLICKDATA"] div[class="title"] a')])
            print('\r正在爬取%d/%d'%(page,totalPage) , end = '')
        # 这里并多线程调用gethouseinfo函数  
        # time1 = time.tme()
        pool = Pool(8)
        houseInfo = pool.map(gethouseInfo,houseUrl)
        pool.close()
        pool.join()
        houseInfo = pd.concat(houseInfo,ignore_index=True)
        # time2 = time.time()
        # print('\n'+str(time2-time1))
        houseInfo.to_csv('houseInfo_perblock.csv',index=False,header=False,mode='a+')
        gc.collect() 
        return houseInfo
    else:
        print('没房，跳过了')
        gc.collect() 

In [2]:
#总地区数
totalBlock = len(blockUrl)
page = 1
#单线程爬取每个地区，看看报错了会怎么样
time3 = time.time()
for eachblock in blockUrl:
    gethouseInfo_perblock(eachblock)
    print('\n %d/%d'%(page,totalBlock))
    page += 1
time4 = time.time()
print('\n 搞定了，总计用时%d秒'%(time4-time3))

NameError: name 'blockUrl' is not defined

In [44]:
blockUrl[1]

'https://bj.lianjia.com/ershoufang/wangjing/'